# SAT_RefSys

   Transformations betweeen celestial and terrestrial reference systems

- EqnEquinox
- GHAMatrix
- Geodetic
- NutMatrix
- IERS 

In [1]:
using NBInclude
using LinearAlgebra

@nbinclude("./SAT_Const.ipynb")
@nbinclude("./SAT_Kepler.ipynb")
@nbinclude("./SAT_VecMat.ipynb")

R_z (generic function with 1 method)

In [24]:
function EqnEquinox(Mjd_TT)
"""    
Purpose: Computation of the equation of the equinoxes
    
Input:
    Mjd_TT    Modified Julian Date (Terrestrial Time)
Output
    Equation of the equinoxes
"""    

# Nutation in longitude and obliquity 
      
  (dpsi,deps)=NutAngles(Mjd_TT)

# Equation of the equinoxes

  return  dpsi*cos(MeanObliquity(Mjd_TT))

end

"""
@nbinclude("./SAT_Time.ipynb")

Mjd0_UTC = Mjd(1999,3,1,0,0,0.0)
EqnEquinox(Mjd0_UTC)
"""

"@nbinclude(\"./SAT_Time.ipynb\")\n\nMjd0_UTC = Mjd(1999,3,1,0,0,0.0)\nEqnEquinox(Mjd0_UTC)\n"

In [25]:
function GMST(Mjd_UT1)

  Secs = 86400.0        # Seconds per day

  # Mean Sidereal Time
  
  Mjd_0 = floor(Mjd_UT1)
  UT1   = Secs*(Mjd_UT1-Mjd_0)          # [s]
  T_0   = (Mjd_0  -MJD_J2000)/36525.0 
  T     = (Mjd_UT1-MJD_J2000)/36525.0;

  gmst  = 24110.54841 + 8640184.812866*T_0 + 1.002737909350795*UT1
          + (0.093104-6.2e-6*T)*T*T; # [s]

  return  pi2*Frac(gmst/Secs);       # [rad], 0..2pi

end

"""
@nbinclude("./SAT_Time.ipynb")

Mjd0_UTC = Mjd(1999,3,1,0,0,0.0)
GMST(Mjd0_UTC)
"""

GMST (generic function with 1 method)

In [29]:
function GAST(Mjd_UT1)
"""
  Purpose: 
     Greenwich Apparent Sidereal Time
  Input:
     Mjd_UT1 : Modified Julian Date UT1
  Output:
     GMST in [rad]
"""
  return mod(GMST(Mjd_UT1) + EqnEquinox(Mjd_UT1),pi2)

end

GAST (generic function with 1 method)

In [33]:
function GHAMatrix(Mjd_UT1)
    A=zeros(3,3)
    R_z(A,GAST(Mjd_UT1))
    return  A
end    
"""
@nbinclude("./SAT_Time.ipynb")

Mjd0_UTC = Mjd(1999,3,1,0,0,0.0)
GHAMatrix(Mjd0_UTC)
"""

"@nbinclude(\"./SAT_Time.ipynb\")\n\nMjd0_UTC = Mjd(1999,3,1,0,0,0.0)\nGHAMatrix(Mjd0_UTC)\n"

## Geodetic

In [3]:
# Constructor for geodetic coordinates from given position

function Geodetic(r,                         # Position vector [m]
                   R_equ,                     # Equator radius [m]
                   f)                         # Flattening

  eps     = 1.0e3*eps_mach   # Convergence criterion 
  epsRequ = eps*R_equ
  e2      = f*(2.0-f)        # Square of eccentricity
  
  X = r[1]                   # Cartesian coordinates
  Y = r[2]
  Z = r[3]
  rho2 = X*X + Y*Y           # Square of distance from z-axis
  
  # Check validity of input data

  if (norm(r)==0.0) 
    println(" invalid input in Geodetic constructor")
    return nothing
  end

  # Iteration 

  dZ = e2*Z
  dZ_new=-1.
  ZdZ=Z
  Nh=0.
  N=0.
 

  while abs(dZ-dZ_new) > epsRequ
    ZdZ    =  Z + dZ
    Nh     =  sqrt( rho2 + ZdZ*ZdZ ) 
    SinPhi =  ZdZ / Nh                    # Sine of geodetic latitude
    N      =  R_equ / sqrt(1.0-e2*SinPhi*SinPhi)
    dZ_new =  N*e2*SinPhi
    dZ = dZ_new
  end
    
  # Longitude, latitude, altitude

  lon = atan( Y, X )
  lat = atan( ZdZ, sqrt(rho2) )
  h   = Nh - N

  return lon,lat,h

end

"""
r=[100.,100.,100.]
R_Earth=6378.137e3
f_Earth=1.0/298.257223563

Geodetic(r,R_Earth,f_Earth)  

"""


"r=[100.,100.,100.]\nR_Earth=6378.137e3\nf_Earth=1.0/298.257223563\n\nGeodetic(r,R_Earth,f_Earth)  \n\n"

## NutMatrix

In [18]:
function MeanObliquity(Mjd_TT)
"""
Purpose:
    Computes the mean obliquity of the ecliptic
Input:
    Modified Julian Date (Terrestrial Time)
Output:
    Mean obliquity of the ecliptic
"""
  T = (Mjd_TT-MJD_J2000)/36525.0

  return  Rad*( 23.43929111-(46.8150+(0.00059-0.001813*T)*T)*T/3600.0 )
end

"""
nire testa
@nbinclude("./SAT_Time.ipynb")

Mjd0_UTC = Mjd(1999,3,1,0,0,0.0)
MeanObliquity(Mjd0_UTC)

"""

"nire testa\n@nbinclude(\"./SAT_Time.ipynb\")\n\nMjd0_UTC = Mjd(1999,3,1,0,0,0.0)\nMeanObliquity(Mjd0_UTC)\n\n"

In [19]:
function EclMatrix(Mjd_TT)
"""
Purpose:
      Transformation of equatorial to ecliptical coordinates 
    
Input/Output:
      Mjd_TT    Modified Julian Date (Terrestrial Time)
      <return>  Transformation matrix
"""    
  A=zeros(3,3)
  R_x(A,MeanObliquity(Mjd_TT))
  return A
end


function PrecMatrix(Mjd_1,Mjd_2)
"""
Purpose:
    Precession transformation of equatorial coordinates 

Input/Output:
     Mjd_1     Epoch given (Modified Julian Date TT)
     MjD_2     Epoch to precess to (Modified Julian Date TT)
     <return>  Precession transformation matrix

"""    
   T  = (Mjd_1-MJD_J2000)/36525.0
   dT = (Mjd_2-Mjd_1)/36525.0
  
#  Precession angles
  
  zeta  =  ( (2306.2181+(1.39656-0.000139*T)*T)+
                ((0.30188-0.000344*T)+0.017998*dT)*dT )*dT/Arcs
  z     =  zeta + ( (0.79280+0.000411*T)+0.000205*dT)*dT*dT/Arcs
  theta =  ( (2004.3109-(0.85330+0.000217*T)*T)-
                ((0.42665+0.000217*T)+0.041833*dT)*dT )*dT/Arcs

#  Precession matrix
  
  A1=zeros(3,3)
  A2=zeros(3,3)
  A3=zeros(3,3)
  R_z(A1,-z)
  R_y(A2,theta)
  R_z(A3,-zeta)
  return A1*A2*A3    

end   

"""
nire testa

@nbinclude("./SAT_Time.ipynb")

Mjd0_UTC = Mjd(1999,3,1,0,0,0.0)
PrecMatrix(MJD_J2000,Mjd0_UTC)

"""

"nire testa\n\n@nbinclude(\"./SAT_Time.ipynb\")\n\nMjd0_UTC = Mjd(1999,3,1,0,0,0.0)\nPrecMatrix(MJD_J2000,Mjd0_UTC)\n\n"

In [20]:
function NutAngles(Mjd_TT)
"""
  Purpose:
    Nutation in longitude and obliquity

  Input:
    Mjd_TT    Modified Julian Date (Terrestrial Time)
   
  Output:
    Nutation Angles
"""
    
  T  = (Mjd_TT-MJD_J2000)/36525.0;
  T2 = T*T;
  T3 = T2*T;
  rev = 360.0*3600.0;  #arcsec/revolution

  N_coeff = 106
    
 C=[
#     l  l' F  D Om    dpsi    *T      deps     *T       #
      0  0  0  0  1  -1719960 -1742   920250   89;      
      0  0  0  0  2     20620     2    -8950    5;
     -2  0  2  0  1       460     0     -240    0; 
      2  0 -2  0  0       110     0        0    0; 
     -2  0  2  0  2       -30     0       10    0;
      1 -1  0 -1  0       -30     0        0    0; 
      0 -2  2 -2  1       -20     0       10    0; 
      2  0 -2  0  1        10     0        0    0; 
      0  0  2 -2  2   -131870   -16    57360  -31;
      0  1  0  0  0     14260   -34      540   -1; 
      0  1  2 -2  2     -5170    12     2240   -6; 
      0 -1  2 -2  2      2170    -5     -950    3; 
      0  0  2 -2  1      1290     1     -700    0; 
      2  0  0 -2  0       480     0       10    0;
      0  0  2 -2  0      -220     0        0    0;
      0  2  0  0  0       170    -1        0    0;
      0  1  0  0  1      -150     0       90    0;
      0  2  2 -2  2      -160     1       70    0;
      0 -1  0  0  1      -120     0       60    0;
     -2  0  0  2  1       -60     0       30    0;
      0 -1  2 -2  1       -50     0       30    0;
      2  0  0 -2  1        40     0      -20    0;
      0  1  2 -2  1        40     0      -20    0;
      1  0  0 -1  0       -40     0        0    0;
      2  1  0 -2  0        10     0        0    0;
      0  0 -2  2  1        10     0        0    0;
      0  1 -2  2  0       -10     0        0    0;
      0  1  0  0  2        10     0        0    0;
     -1  0  0  1  1        10     0        0    0;
      0  1  2 -2  0       -10     0        0    0;
      0  0  2  0  2    -22740    -2     9770   -5;
      1  0  0  0  0      7120     1      -70    0;
      0  0  2  0  1     -3860    -4     2000    0;
      1  0  2  0  2     -3010     0     1290   -1;
      1  0  0 -2  0     -1580     0      -10    0;
     -1  0  2  0  2      1230     0     -530    0;
      0  0  0  2  0       630     0      -20    0;
      1  0  0  0  1       630     1     -330    0;
     -1  0  0  0  1      -580    -1      320    0;
     -1  0  2  2  2      -590     0      260    0;
      1  0  2  0  1      -510     0      270    0;
      0  0  2  2  2      -380     0      160    0;
      2  0  0  0  0       290     0      -10    0;
      1  0  2 -2  2       290     0     -120    0;
      2  0  2  0  2      -310     0      130    0;
      0  0  2  0  0       260     0      -10    0;
     -1  0  2  0  1       210     0     -100    0;
     -1  0  0  2  1       160     0      -80    0;
      1  0  0 -2  1      -130     0       70    0;
     -1  0  2  2  1      -100     0       50    0;
      1  1  0 -2  0       -70     0        0    0;
      0  1  2  0  2        70     0      -30    0;
      0 -1  2  0  2       -70     0       30    0;
      1  0  2  2  2       -80     0       30    0;
      1  0  0  2  0        60     0        0    0;
      2  0  2 -2  2        60     0      -30    0;
      0  0  0  2  1       -60     0       30    0;
      0  0  2  2  1       -70     0       30    0;
      1  0  2 -2  1        60     0      -30    0;
      0  0  0 -2  1       -50     0       30    0;
      1 -1  0  0  0        50     0        0    0;
      2  0  2  0  1       -50     0       30    0;
      0  1  0 -2  0       -40     0        0    0;
      1  0 -2  0  0        40     0        0    0;
      0  0  0  1  0       -40     0        0    0;
      1  1  0  0  0       -30     0        0    0;
      1  0  2  0  0        30     0        0    0;
      1 -1  2  0  2       -30     0       10    0;
     -1 -1  2  2  2       -30     0       10    0;
     -2  0  0  0  1       -20     0       10    0;
      3  0  2  0  2       -30     0       10    0;
      0 -1  2  2  2       -30     0       10    0;
      1  1  2  0  2        20     0      -10    0;
     -1  0  2 -2  1       -20     0       10    0;
      2  0  0  0  1        20     0      -10    0;
      1  0  0  0  2       -20     0       10    0;
      3  0  0  0  0        20     0        0    0;
      0  0  2  1  2        20     0      -10    0;
     -1  0  0  0  2        10     0      -10    0;
      1  0  0 -4  0       -10     0        0    0;
     -2  0  2  2  2        10     0      -10    0;
     -1  0  2  4  2       -20     0       10    0;
      2  0  0 -4  0       -10     0        0    0;
      1  1  2 -2  2        10     0      -10    0;
      1  0  2  2  1       -10     0       10    0;
     -2  0  2  4  2       -10     0       10    0;
     -1  0  4  0  2        10     0        0    0;
      1 -1  0 -2  0        10     0        0    0;
      2  0  2 -2  1        10     0      -10    0;
      2  0  2  2  2       -10     0        0    0;
      1  0  0  2  1       -10     0        0    0;
      0  0  4 -2  2        10     0        0    0;
      3  0  2 -2  2        10     0        0    0;
      1  0  2 -2  0       -10     0        0    0;
      0  1  2  0  1        10     0        0    0;
     -1 -1  0  2  1        10     0        0    0;
      0  0 -2  0  1       -10     0        0    0;
      0  0  2 -1  2       -10     0        0    0;
      0  1  0  2  0       -10     0        0    0;
      1  0 -2 -2  0       -10     0        0    0;
      0 -1  2  0  1       -10     0        0    0;
      1  1  0 -2  1       -10     0        0    0;
      1  0 -2  2  0       -10     0        0    0;
      2  0  0  2  0        10     0        0    0;
      0  0  2  4  2       -10     0        0    0;
      0  1  0  1  0        10     0        0    0;
   ]


# Mean arguments of luni-solar motion
#
#   l   mean anomaly of the Moon
#   l'  mean anomaly of the Sun
#   F   mean argument of latitude
#   D   mean longitude elongation of the Moon from the Sun 
#   Om  mean longitude of the ascending node
  
  l  = mod(485866.733 + (1325.0*rev +  715922.633)*T    
                                    + 31.310*T2 + 0.064*T3,rev)
  lp = mod(1287099.804 + (99.0*rev + 1292581.224)*T             
                                   - 0.577*T2 - 0.012*T3, rev )
  F  = mod(335778.877 + (1342.0*rev + 295263.137)*T    
                                    - 13.257*T2 + 0.011*T3,rev)
  D  = mod(1072261.307 + (1236.0*rev +1105601.328)*T    
                                     - 6.891*T2 + 0.019*T3,rev)
  Om = mod(450160.280  - (5.0*rev + 482890.539)*T    
                                  +7.455*T2 + 0.008*T3, rev )

# Nutation in longitude and obliquity [rad]

  deps = dpsi = 0.0
  for i in 1:N_coeff
    arg  =  ( C[i,1]*l+C[i,2]*lp+C[i,3]*F+C[i,4]*D+C[i,5]*Om )/Arcs
    dpsi += ( C[i,6]+C[i,7]*T)*sin(arg)
    deps += ( C[i,8]+C[i,9]*T)*cos(arg)
  end
      
  dpsi = 1.0E-5*dpsi/Arcs
  deps = 1.0E-5*deps/Arcs

  return dpsi,deps  
    
end

"""
nire testa

@nbinclude("./SAT_Time.ipynb")

Mjd0_UTC = Mjd(1999,3,1,0,0,0.0)
(dpsi, deps)=NutAngles(Mjd0_UTC)

"""

"nire testa\n\n@nbinclude(\"./SAT_Time.ipynb\")\n\nMjd0_UTC = Mjd(1999,3,1,0,0,0.0)\n(dpsi, deps)=NutAngles(Mjd0_UTC)\n\n"

In [21]:
function NutMatrix(Mjd_TT)
"""
Purpose:
   Transformation from mean to true equator and equinox
Input:
   Mjd_TT    Modified Julian Date (Terrestrial Time)
Output:    
   Nutation matrix 
"""
  
  # Mean obliquity of the ecliptic

  eps = MeanObliquity(Mjd_TT)

  # Nutation in longitude and obliquity

  (dpsi,deps)=NutAngles(Mjd_TT)

  # Transformation from mean to true equator and equinox

  A1=zeros(3,3)
  A2=zeros(3,3)
  A3=zeros(3,3)
  R_x(A1,-eps-deps)
  R_z(A2,-dpsi)
  R_x(A3,+eps)
    
  return  A1*A2*A3

end

"""
testa
@nbinclude("./SAT_Time.ipynb")

Mjd0_UTC = Mjd(1999,3,1,0,0,0.0)
NutMatrix(Mjd0_UTC)
"""

"testa\n@nbinclude(\"./SAT_Time.ipynb\")\n\nMjd0_UTC = Mjd(1999,3,1,0,0,0.0)\nNutMatrix(Mjd0_UTC)\n"

## IERS

In [ ]:
"""
struct IERS{T}
    TT_TAI::T
    GSP_TAI::T
    TT_GPS::T
    TAI_GPS::T
    UT1_TAI_::T
    UTC_TAI_::T
    x_pole_::T
    y_pole_::T
end
"""

In [2]:
function IERS_TT_UTC(Mjd_UTC,UTC_TAI_)
"""   
double IERS::TT_UTC (double Mjd_UTC){return  TT_TAI-UTC_TAI(Mjd_UTC);};
       double IERS::UTC_TAI(double Mjd_UTC){return UTC_TAI_;};    
"""
  TT_TAI  = +32.184 
    
  return  TT_TAI-UTC_TAI_ 

end

function IERS_UT1_UTC(Mjd_UTC,UT1_TAI_,UTC_TAI_)
"""
  double IERS::UT1_UTC(double Mjd_UTC){return UT1_TAI(Mjd_UTC)-UTC_TAI(Mjd_UTC);};  
         double IERS::UT1_TAI(double Mjd_UTC){return UT1_TAI_;}; 
         double IERS::UTC_TAI(double Mjd_UTC){return UTC_TAI_;};
"""
   
   return(UT1_TAI_-UTC_TAI_)
    
end

IERS_UT1_UTC (generic function with 1 method)